In [1]:
import json
import glob
import sys, os, shutil
import numpy as np
from tqdm.auto import tqdm 

sys.path.append('/home/daniel/research/catkin_ws/src/')
from grub_nempc import grub_nempc

In [ ]:
iae_metrics = []
trial_names = []
INPUT_DIR = '/home/daniel/research/catkin_ws/src/hyperparam_optimization/bellows_grub/run_logs/'

# run_logs = os.listdir(INPUT_DIR)
run_logs = ['fnn_testrun']

for run in run_logs:
    for trial in os.listdir(INPUT_DIR+run):
        if os.path.isdir(INPUT_DIR + run + '/' + trial):
            trial_dir = INPUT_DIR + run + '/' + trial
            trial_name = trial[:14]

            try:
                checkpoint_path = glob.glob(trial_dir + '/lightning_logs/version_0/checkpoints/*.ckpt')[0] # get the checkpoint

                with open(trial_dir + '/params.json', 'r') as f:
                    config = json.load(f) # get the config file

                print(f'\n\nRunning Control for Trial: {trial_name}')

                # Try the controller with the model============
                test_runs = 10
                control_results = np.zeros([test_runs,])
                iae_results = np.zeros([test_runs,])

                # Initial Conditions
                x0 = np.array([200,200,200,200,0,0,0.1,0.1]).reshape(8,1) # p, qd, q
                u0 = 200*np.ones([4, 1])
                p_goal = np.array([[0], [0], [0], [0]])
                qd_goal = np.zeros((2,1))
                ugoal = np.zeros([4, 1]) # right now in the cost function this is weighted at 0

                for test in tqdm(range(test_runs)):
                    q_goal = np.random.uniform(low=-np.pi/2, high=np.pi/2, size=(2, 1)) # random joint angle goal
                    xgoal = np.vstack([p_goal,qd_goal,q_goal])

                    x, iae_score, control_successful = grub_nempc(checkpoint_path, config, x0, u0, xgoal, ugoal, visualize=False, plot=False)
                    control_results[test] = control_successful
                    iae_results[test] = iae_score
                    # print(f'u=({q_goal[0]},{x[6]}) and v=({q_goal[1]},{x[7]})')
                # Try the controller with the model ==============================
                
                iae_metrics.append(np.mean(iae_results))
                trial_names.append(trial_name+'/'+run)
                if np.mean(control_results) >= 0.5:
                    print(f'Ran control successfully for trial: {trial_name}. Control Results: {np.mean(control_results)}, Average IAE={np.mean(iae_results)}')
                else:
                    print(f'Control failed for trial: {trial_name}. Control Results: {np.mean(control_results)}, Average IAE={np.mean(iae_results)}')
                    shutil.rmtree(trial_dir) # delete the trial directory since there is no checkpoint to use


            except Exception as e:
                print(f'For trial {trial_name} got {e}')
                shutil.rmtree(trial_dir)
                continue # no checkpoint, wasn't a good run

min_index = iae_metrics.index(min(iae_metrics))
print()
print(f'Best Trial: {trial_names[min_index]}, with IAE={iae_metrics[min_index]}')
print(f'Total Trials ran: {len(iae_metrics)}')

In [3]:
trial_names = []
INPUT_DIR = '/home/daniel/research/catkin_ws/src/hyperparam_optimization/bellows_grub/run_logs/'

# run_logs = os.listdir(INPUT_DIR)
run_logs = ['fnn_testrun']

for run in run_logs:
    for trial in os.listdir(INPUT_DIR+run):
        if os.path.isdir(INPUT_DIR + run + '/' + trial):
            trial_dir = INPUT_DIR + run + '/' + trial
            trial_name = trial[:14]

            checkpoint_path = glob.glob(trial_dir + '/lightning_logs/version_0/checkpoints/*.ckpt')[0] # get the checkpoint

            with open(trial_dir + '/params.json', 'r') as f:
                config = json.load(f) # get the config file

            print(f'\n\nRunning Control for Trial: {trial_name}')

            # Try the controller with the model============
            test_runs = 50
            control_results = np.zeros([test_runs,])
            iae_results = np.zeros([test_runs,])

            p0 = np.array([[200],[200],[200],[200]])
            qd0 = np.zeros((2,1))
            q0 = .1*np.ones((2,1)) #np.random.uniform(low=-np.pi/2, high=np.pi/2, size=(2, 1)) # random joint angle starting location
            x0 = np.vstack([p0,qd0,q0])
            u0 = 200*np.ones([4, 1])

            p_goal = np.array([[0], [0], [0], [0]])
            qd_goal = np.zeros((2,1))
            ugoal = np.zeros([4, 1]) # right now in the cost function this is weighted at 0

            for test in tqdm(range(test_runs)):
                q_goal = np.random.uniform(low=-np.pi/2, high=np.pi/2, size=(2, 1)) # random joint angle goal
                xgoal = np.vstack([p_goal,qd_goal,q_goal])
                x, iae_score, control_successful = grub_nempc(checkpoint_path, config, x0, u0, xgoal, ugoal, visualize=False, plot=False)
                control_results[test] = control_successful
                iae_results[test] = iae_score
                # print(f'u=({q_goal[0]},{x[6]}) and v=({q_goal[1]},{x[7]})')
            # Try the controller with the model ==============================
            
            trial_names.append(trial_name+'/'+run)
            if np.mean(control_results) >= 0.5:
                print(f'Ran control successfully for trial: {trial_name}. Control Results: {np.mean(control_results)}, Average IAE={np.mean(iae_results)}')
            else:
                print(f'Control failed for trial: {trial_name}. Control Results: {np.mean(control_results)}, Average IAE={np.mean(iae_results)}')



Running Control for Trial: train_13c2219f


  0%|          | 0/50 [00:00<?, ?it/s]

Control failed for trial: train_13c2219f. Control Results: 0.14, Average IAE=240.57370967271578


Running Control for Trial: train_9b194ef8


  0%|          | 0/50 [00:00<?, ?it/s]

Control failed for trial: train_9b194ef8. Control Results: 0.28, Average IAE=220.38932284517716


Running Control for Trial: train_aea2950b


...

Control failed for trial: train_e8d1e215. Control Results: 0.1, Average IAE=226.12273268394844


Running Control for Trial: train_d923b9e2


  0%|          | 0/50 [00:00<?, ?it/s]

Control failed for trial: train_d923b9e2. Control Results: 0.1, Average IAE=250.21463069716262


Running Control for Trial: train_e2268250


  0%|          | 0/50 [00:00<?, ?it/s]

KeyboardInterrupt: 